# 필요 모듈 임포트

In [ ]:
import json
import os
import pandas as pd
import warnings
from tqdm import tqdm
import numpy as np

warnings.filterwarnings('ignore')

# 1. COCO 형식 bbox를 YOLO 형식 bbox로 변환하는 함수

In [ ]:
def transform(bbox, w, h):
    box = np.array(bbox, dtype=np.float64)
    box[:2] += box[2:] / 2 # COCO 형식의 어노테이션 파일의 x,y는 좌측상단
    box[[0, 2]] /= w  # x 정규화 
    box[[1, 3]] /= h  # y 정규화
    
    return box

# 2. YOLOv8 모델을 위한 어노테이션 txt 파일 생성

In [ ]:
#어노테이션 파일을 생성할 이미지들이 있는 디렉토리
image_path = './images'
#COCO 형식의 json 파일이 있는 디렉토리
json_path = './annotations'
#YOLO 형식의 txt 파일을 저장할 디렉토리
txt_path = './labels'

#각 파손 유형별 json 파일과 각 파손 유형별 인덱스
json_dict = {'균열-길이(Crack-Longitudinal,CL).json' : 0,
            '균열-원주(Crack-Circumferential,CC).json':1,
            '표면손상(Surface-Damage,SD).json':2,
            '파손(Broken-Pipe,BK).json':3,
            '연결관-돌출(Lateral-Protruding,LP).json':4,
            '이음부-손상(Joint-Faulty,JF).json':5,
            '이음부-단차(Joint-Displaced,JD).json':6,
            '토사퇴적(Deposits-Silty,DS).json':7,
            '기타결함(Etc.,ETC).json':8}

In [ ]:
exif = 0

for json_file in tqdm(json_dict):
    
    #json 어노테이션 파일 로드
    with open(json_path + '/' + json_file) as f:
        data = json.load(f)
        
    #EXIF는 매우 적으므로 무시
    for cat in data['categories']:
        if cat['name'] == 'EX':
            exif = cat['id']
    
    #클래스 인덱스
    class_id = json_dict[json_file]
    #images 정보가 담긴 데이터프레임 형성
    image_df = pd.DataFrame(data = data['images'])
    #annotations 정보가 담긴 데이터프레임 형성
    annot_df = pd.DataFrame(data = data['annotations'])
    #images 데이터프레임과 annotations 데이터프레임 병합
    df = pd.merge(left = image_df, right = annot_df, left_on = 'id', right_on = 'image_id')
    #클래스 인덱스로 category_id 변경
    df['category_id'] = class_id
    #인덱스 리셋
    df.reset_index(inplace = True, drop = True)

    #bbox 변경
    for i in range(len(df)):
        bbox  = df.loc[i, 'bbox']
        height = df.loc[i, 'height']
        width = df.loc[i, 'width']
        bbox = scaling(bbox, width, height) 
        df.loc[i, 'bbox'] = str(bbox)
    #문자열 리스트 타입으로 변경
    df['bbox'] = df['bbox'].apply(eval)
    
    #file명 기준으로 group by 객체 생성
    grouped = df.groupby('file_name')
    file_names = df['file_name'].unique()
    
    #annotation 텍스트 파일 생성
    for file_name in file_names:
        if file_name[:-4]+'.jpg' in images:
            bboxs = grouped.get_group(file_name)['bbox']
            with open(txt_path + '/' + file_name[:-4] + '.txt', 'w') as t:
                for i, bbox in enumerate(bboxs):
                    line = str(class_id) + ' ' + " ".join(map(str, bbox))
                    if i < len(bboxs) - 1:
                        line += '\n'  # 마지막 bbox가 아닌 경우에만 개행문자 추가
                    t.write(line)